In [2]:
import pandas as pd
import numpy as np
from itertools import product
import math


In [8]:
activations_list = [[1,0,1,0], [1,0,1,0], [0,1,0,0], [0,1,1,1], [1,1,1,0], [0,0,1,1], [1,0,0,1], [1,0,1,1]]
activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
activations_matrix.values

array([[1, 0, 1, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 1, 0],
       [0, 0, 1, 1],
       [1, 0, 0, 1],
       [1, 0, 1, 1]], dtype=int64)

In [12]:
neurons_corr = activations_matrix.corr().values
neurons_corr.shape

(4, 4)

In [13]:
neurons_average = activations_matrix.mean().values
neurons_average

array([0.625, 0.375, 0.75 , 0.5  ])

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [15]:
np.random.seed(3)
J = np.random.normal(loc=0.0, scale=.01, size=neurons_corr.shape)
#np.fill_diagonal(J,0)
lind = np.tril_indices(4)
J[lind]=0
J

array([[ 0.        ,  0.0043651 ,  0.00096497, -0.01863493],
       [ 0.        ,  0.        , -0.00082741, -0.00627001],
       [ 0.        ,  0.        ,  0.        ,  0.00884622],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [17]:
H = activations_matrix.sum()/activations_matrix.shape[0]

sigma_0    0.625
sigma_1    0.375
sigma_2    0.750
sigma_3    0.500
dtype: float64

In [20]:
J

array([[ 0.        ,  0.0043651 ,  0.00096497, -0.01863493],
       [ 0.        ,  0.        , -0.00082741, -0.00627001],
       [ 0.        ,  0.        ,  0.        ,  0.00884622],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [21]:
H

sigma_0    0.625
sigma_1    0.375
sigma_2    0.750
sigma_3    0.500
dtype: float64

In [22]:
spin_values = [-1,1]

# Calcular las probabilidades
# Se pueden iterar todos los valores antes y guardarlos en un diccionario
prob_dict = {}
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    prob_dict[sigma] = 0
    for i in range(0,activations_matrix.shape[1]):
        for j in range(i+1,activations_matrix.shape[1]):
            prob_dict[sigma] += sigma[i]*sigma[j]*J[i][j]
            
    for i in range(0,activations_matrix.shape[1]):
        prob_dict[sigma] += sigma[i]*H[i]
    
    prob_dict[sigma] = math.exp(prob_dict[sigma])
            
Z = sum(prob_dict.values())

for sigma in prob_dict.keys():
    prob_dict[sigma] = prob_dict[sigma]/Z


In [23]:
corr_model_exp = np.zeros(J.shape)
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    for i in range(0,activations_matrix.shape[1]):
        for j in range(i+1,activations_matrix.shape[1]):
            print(i,j)
            corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]
    break
corr_model_exp

0 1
0 2
0 3
1 2
1 3
2 3


array([[0.        , 0.00347207, 0.00347207, 0.00347207],
       [0.        , 0.        , 0.00347207, 0.00347207],
       [0.        , 0.        , 0.        , 0.00347207],
       [0.        , 0.        , 0.        , 0.        ]])

In [11]:
mean_model_exp = np.zeros(H.shape)
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    for i in range(0,activations_matrix.shape[1]):
        mean_model_exp[i] += sigma[i]*prob_dict[sigma]
mean_model_exp

array([0.5476106 , 0.36278193, 0.62971033, 0.46116611])

In [39]:
class IsingModel:
    def train(self, data ,lr=10**-4):
        np.random.seed(3)
        
        activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        spins_correlation = activations_matrix.corr().values
        lind = np.tril_indices_from(spins_correlation)
        spins_correlation[lind]=0
        spins_average = activations_matrix.mean().values

        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        J = np.random.normal(loc=0.0, scale=.01, size=spins_correlation.shape)
        lind = np.tril_indices_from(J)
        J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        H = activations_matrix.sum()/activations_matrix.shape[0]
        
        totalParamVariation = math.inf
        stopCondition = 0.5
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 1000:
            # Calculate P(sigma) for every possible combination in the model
            spin_values = [-1,1]
            prob_dict = {}
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                prob_dict[sigma] = 0
                for i in range(0,activations_matrix.shape[1]):
                    for j in range(i+1,activations_matrix.shape[1]):
                        prob_dict[sigma] += sigma[i]*sigma[j]*J[i][j]

                for i in range(0,activations_matrix.shape[1]):
                    prob_dict[sigma] += sigma[i]*H[i]

                prob_dict[sigma] = math.exp(prob_dict[sigma])

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            # Calculate the expectation of the correlations
            corr_model_exp = np.zeros(J.shape)
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                for i in range(0,activations_matrix.shape[1]):
                    for j in range(i+1,activations_matrix.shape[1]):
                        corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

            # Calculate the expectation of the averages
            mean_model_exp = np.zeros(H.shape)
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                for i in range(0,activations_matrix.shape[1]):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = lr * (spins_correlation - corr_model_exp)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(J - stepJ).sum().sum()
            # Take the step
            J = J + stepJ

            # Calculate the step size for every H_{i}
            stepH = lr * (spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(H - stepH).sum()
            # Take the step
            H = H + stepH
            totalParamVariation = 0
            totalParamVariation = deltaH + deltaJ
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(H)
        print(spins_correlation)
        print(J)

In [40]:
activations_list = [[1,0,1,0], [1,0,1,0], [0,1,0,0], [0,1,1,1], [1,1,1,0], [0,0,1,1], [1,0,0,1], [1,0,1,1]]


ising = IsingModel()
ising.train(activations_list)

Epoch 1 TotalParamVariation 2.289870888410531
Epoch 2 TotalParamVariation 2.289908632520788
Epoch 3 TotalParamVariation 2.2899463897221763
Epoch 4 TotalParamVariation 2.2899841600103192
Epoch 5 TotalParamVariation 2.2900219433808413
Epoch 6 TotalParamVariation 2.290059739829368
Epoch 7 TotalParamVariation 2.2900975493515268
Epoch 8 TotalParamVariation 2.2901353719429474
Epoch 9 TotalParamVariation 2.29017320759926
Epoch 10 TotalParamVariation 2.290211056316097
Epoch 11 TotalParamVariation 2.2902489180890915
Epoch 12 TotalParamVariation 2.290286792913879
Epoch 13 TotalParamVariation 2.290324680786096
Epoch 14 TotalParamVariation 2.290362581701381
Epoch 15 TotalParamVariation 2.2904004956553727
Epoch 16 TotalParamVariation 2.290438422643714
Epoch 17 TotalParamVariation 2.2904763626620452
Epoch 18 TotalParamVariation 2.290514315706013
Epoch 19 TotalParamVariation 2.2905522817712614
Epoch 20 TotalParamVariation 2.2905902608534383
Epoch 21 TotalParamVariation 2.290628252948192
Epoch 22 Tota

Epoch 345 TotalParamVariation 2.3530167489927076
Epoch 346 TotalParamVariation 2.353279704746216
Epoch 347 TotalParamVariation 2.353542639692528
Epoch 348 TotalParamVariation 2.35380555383728
Epoch 349 TotalParamVariation 2.35406844718611
Epoch 350 TotalParamVariation 2.3543313197446514
Epoch 351 TotalParamVariation 2.3545941715185363
Epoch 352 TotalParamVariation 2.354857002513396
Epoch 353 TotalParamVariation 2.3551198127348605
Epoch 354 TotalParamVariation 2.355382602188555
Epoch 355 TotalParamVariation 2.3556453708801057
Epoch 356 TotalParamVariation 2.3559081188151367
Epoch 357 TotalParamVariation 2.35617084599927
Epoch 358 TotalParamVariation 2.3564335524381255
Epoch 359 TotalParamVariation 2.356696238137321
Epoch 360 TotalParamVariation 2.356958903102474
Epoch 361 TotalParamVariation 2.3572215473391984
Epoch 362 TotalParamVariation 2.3574841708531076
Epoch 363 TotalParamVariation 2.357746773649813
Epoch 364 TotalParamVariation 2.358009355734925
Epoch 365 TotalParamVariation 2.35

Epoch 667 TotalParamVariation 2.436643472455528
Epoch 668 TotalParamVariation 2.4369000100168705
Epoch 669 TotalParamVariation 2.437156528496532
Epoch 670 TotalParamVariation 2.4374130278995954
Epoch 671 TotalParamVariation 2.4376695082311457
Epoch 672 TotalParamVariation 2.437925969496263
Epoch 673 TotalParamVariation 2.438182411700028
Epoch 674 TotalParamVariation 2.4384388348475183
Epoch 675 TotalParamVariation 2.4386952389438123
Epoch 676 TotalParamVariation 2.438951623993982
Epoch 677 TotalParamVariation 2.439207990003104
Epoch 678 TotalParamVariation 2.4394643369762474
Epoch 679 TotalParamVariation 2.4397206649184833
Epoch 680 TotalParamVariation 2.4399769738348804
Epoch 681 TotalParamVariation 2.4402332637305046
Epoch 682 TotalParamVariation 2.440489534610421
Epoch 683 TotalParamVariation 2.4407457864796935
Epoch 684 TotalParamVariation 2.441002019343384
Epoch 685 TotalParamVariation 2.4412582332065518
Epoch 686 TotalParamVariation 2.4415144280742562
Epoch 687 TotalParamVariatio

Epoch 983 TotalParamVariation 2.5167854731828454
Epoch 984 TotalParamVariation 2.517036225586599
Epoch 985 TotalParamVariation 2.5172869604356287
Epoch 986 TotalParamVariation 2.5175376777345226
Epoch 987 TotalParamVariation 2.517788377487863
Epoch 988 TotalParamVariation 2.5180390597002322
Epoch 989 TotalParamVariation 2.5182897243762117
Epoch 990 TotalParamVariation 2.5185403715203796
Epoch 991 TotalParamVariation 2.5187910011373154
Epoch 992 TotalParamVariation 2.5190416132315927
Epoch 993 TotalParamVariation 2.519292207807789
Epoch 994 TotalParamVariation 2.5195427848704757
Epoch 995 TotalParamVariation 2.5197933444242255
Epoch 996 TotalParamVariation 2.520043886473608
Epoch 997 TotalParamVariation 2.520294411023193
Epoch 998 TotalParamVariation 2.520544918077546
Epoch 999 TotalParamVariation 2.5207954076412347
sigma_0    0.634110
sigma_1    0.379843
sigma_2    0.761896
sigma_3    0.506351
dtype: float64
[[ 0.         -0.46666667  0.1490712  -0.25819889]
 [ 0.          0.         -